## Preprocessing

In [30]:
import pandas as pd
import re
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import json
import pandas
import os
import io
import random
import json
from nltk.corpus import stopwords
import string
import numpy as np
from scipy import stats
from gsitk.preprocess import pprocess_twitter

Read file

In [31]:
# with this file we take the tweets from the database of hate_labeled_data.csv and put in a file, with each line a tweet
data = pd.read_csv('hate_labeled_data.csv')
data = data.rename(columns={'count': 'counter', 'class': 'type'})   # change names so they dont interfere with class names
data = data.sort_values(by='counter', ascending=False)

Preprocess the twits with gsitk tool

In [14]:
data['tweet'] = data['tweet'].map(pprocess_twitter.preprocess)

14007    rt <allcaps> <user>: <user> <user> nah boa tha...
7415     a bitch like you need to get fucked right ever...
13370                                    nikko a gay bitch
11155    i'd rather follow some girls on instagram rath...
20709    she a hoe cause your boyfriend want her? she s...
                               ...                        
8516     confident in one way, pussy in another. so str...
8514     completely off topic, idgaf about that pretty ...
8513     commentators bitch at bird for "sliding" but <...
8512                                comin from a fat bitch
24782    ~~ruffled | ntac eileen dahlia - beautiful col...
Name: tweet, Length: 24783, dtype: object

In [32]:
# divide the dataset into the category it belongs
# we are going to make two tests, one with offensive language and one with hate_speech

# separate datasets into neither, offensive and hate speech
data_hate = data[data.type == 0][['counter', 'type', 'tweet']]
data_offensive = data[data.type == 1][['counter', 'type', 'tweet']]
data_neither = data[data.type == 2][['counter', 'type', 'tweet']]

print('Hate twits', data_hate.shape)    # ->1430
print('Offensive twits', data_offensive.shape)  # -> 19190
print('Neither twits', data_neither.shape)  # -> 4163



Hate twits (1430, 3)
Offensive twits (19190, 3)
Neither twits (4163, 3)


using stratify to use all the information and maintain the proportion

In [88]:
dataset_hate = data_hate.append(data_neither)
dataset_hate_label = dataset_hate['type']
train_hate, test_hate, train_hate_label, test_hate_label = train_test_split(
    dataset_hate['tweet'], dataset_hate_label, test_size = 0.2, shuffle = True, stratify=dataset_hate_label)



In [89]:

train_hate = train_hate.to_frame()
train_hate['label'] = train_hate_label
train_hate.keys()

Index(['tweet', 'label'], dtype='object')

In [90]:
train_hate

,tweet,label
23070,Ya'll take note. There ain't never tornadoes w...,2
7016,@shanetallant he won't bite! We promise he's a...,2
1200,&#8220;@CommonBlackGirI: here&#8217;s to all t...,2
17718,RT @TheRealMugatu: @Deet02 @MikeNapoli25 @954N...,2
18406,RT @__bettyboo: You better pray I don't send y...,0
...,...,...
7756,And the Jew @Deuterostomes favorited my tweet ...,0
5172,@WyattNuckels @grizzboadams shut the fuck up f...,0
5358,@_TriggaPlease_ @kanyewest he's a faggot,0
3716,"@JunDishes elissa would call me trash, vile, e...",2


In [68]:
print(data_t.keys())
print(data_neither.keys())


Index(['counter', 'type', 'tweet'], dtype='object')
Index(['counter', 'type', 'tweet'], dtype='object')


In [ ]:
# # as we have dispair number of tweets for each class, and we want the data to be balanced
# # we will make the two datasets with the minimum of the values as the limiting one
# # concatenate both dataframes into a final one
# dataset_offensive = data_offensive[0:5000].append(data_neither)
# dataset_hate = data_hate.append(data_neither[0:2000])
# # we shuffle it so the order is random and twits offensive and neither are mixed
# dataset_offensive = shuffle(dataset_offensive)
# dataset_hate = shuffle(dataset_hate)
# # divide into train/test sets
# train_offensive, test_offensive = train_test_split(dataset_offensive, test_size=0.2)
# train_hate, test_hate = train_test_split(dataset_hate, test_size=0.2)

Save train files as txt and test files as csv

In [16]:
# create or erase content of file in case it is exists
f = open('offensive_twits_train.txt', 'w+')
for twit in train_offensive['tweet']:
    f.write(twit + '\n')
f.close()

f = open('hate_twits_train.txt', 'w+')
for twit in train_hate['tweet']:
    f.write(twit + '\n')
f.close()

# save test files as csv
test_offensive.to_csv('offensive_twits_test.csv')
test_hate.to_csv('hate_twits_test.csv')

###### Execute 'hatedictionary.py hate_twits_train.txt hate' before continuing with the next part

## Evaluation

In [7]:
import json
import pandas
import os
import io
import random
import json
from nltk.corpus import stopwords
import string
import numpy as np
from scipy import stats
import pandas as pd
from gsitk.preprocess import pprocess_twitter

get the dictionaries

In [8]:
# hate
with open('dictionaries/hate_densify.json') as json_file:
    hate_dict_densify = json.load(json_file)
with open('dictionaries/hate_socialsent.json') as json_file:
    hate_dict_socialsent = json.load(json_file)
    
# offensive
with open('dictionaries/offensive_densify.json') as json_file:
    offensive_dict_densify = json.load(json_file)
with open('dictionaries/offensive_socialsent.json') as json_file:
    offensive_dict_socialsent = json.load(json_file)

In [9]:
max(list(hate_dict_densify.values()))

7.525424957275391

Function to normalize the dictionary between -1 and 1 values

In [132]:
# def normalize_dict(dictionary):

#     value_list = list(dictionary.values())
#     key_list = list(dictionary.keys())
#     max_list = max(value_list)
#     min_list = min(value_list)
#     a = {key_list[i]: 2 * (value_list[i] - min_list)/(max_list - min_list) -1  for i in range(len(key_list))}
#     return a

In [133]:
# # hate
# hate_dict_socialsent = normalize_dict(hate_dict_socialsent)
# hate_dict_densify = normalize_dict(hate_dict_densify)

# # offensive
# offensive_dict_socialsent = normalize_dict(offensive_dict_socialsent)
# offensive_dict_densify = normalize_dict(offensive_dict_densify)

Read each file and calculate the mean value

In [11]:
hate_twits = pd.read_csv('hate_twits_test.csv')
offensive_twits = pd.read_csv('offensive_twits_test.csv')
hate_twits = hate_twits.drop(columns='Unnamed: 0')

In [12]:
# initialize arrays for evaluation
# ground truth
hate_label = np.zeros(len(hate_twits))
offensive_label = np.zeros(len(offensive_twits))
# evaluated value
hate_eval = np.zeros(len(hate_twits))
hate_offensive = np.zeros(len(offensive_twits))

In [13]:
def evaluate(data, args):
    evaluation_soc = 0  # socialsent
    evaluation_dens = 0   # densify
    counter = 0
    if args == 'hate':
        dict_soc = hate_dict_socialsent
        dict_dens = hate_dict_densify
    elif args == 'offensive':
        dict_soc = offensive_dict_socialsent
        dict_dens = offensive_dict_densify
    for twit in data['tweet']:
        if dict_soc.get(twit) is not None: 
            evaluation_soc += dict_soc.get(twit)
            evaluation_dens += dict_dens.get(twit)
            counter += 1
    if counter == 0:
        counter = 1   # avoid division by 0
    
    data['evaluation_soc'] = 1 if evaluation_soc/counter >= 0.5 else -1
    data['evaluation_dens'] = 1 if evaluation_dens/counter > 0 else -1
    data['label_eval'] = 1 if data['type'] == 2 else -1   # 1 if type is neither, -1 if is hate/offensive
    return data

In [14]:
list(hate_dict_socialsent.keys())[np.argmin(list(hate_dict_socialsent.values()))]

'wetback'

In [15]:
hate_twits_evaluated = hate_twits.apply(evaluate, args=('hate',), axis=1)
offensive_twits_evaluated = offensive_twits.apply(evaluate, args=('offensive',), axis=1)

In [16]:

hate_eval_socialsent = stats.pearsonr(hate_twits_evaluated['evaluation_soc'], hate_twits_evaluated['label_eval'])
hate_eval_densify = stats.pearsonr(hate_twits_evaluated['evaluation_dens'], hate_twits_evaluated['label_eval'])
offensive_eval_socialsent = stats.pearsonr(offensive_twits_evaluated['evaluation_soc'], offensive_twits_evaluated['label_eval'])
offensive_eval_densify = stats.pearsonr(offensive_twits_evaluated['evaluation_dens'], offensive_twits_evaluated['label_eval'])
print(hate_eval_socialsent)
print(hate_eval_densify)
print(offensive_eval_socialsent)
print(offensive_eval_densify)

(nan, nan)
(nan, nan)
(0.03646837439985302, 0.11857221433772075)
(0.03646837439985302, 0.11857221433772075)


C:\Users\david\Anaconda3\envs\uni\lib\site-packages\scipy\stats\stats.py:3399: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())


In [17]:
hate_twits_evaluated

,counter,type,tweet,evaluation_soc,evaluation_dens,label_eval
0,4,2,<user> el negro?,1,1,1
1,3,0,<user> nigga who's bitch u fucking cracker get...,1,1,-1
2,3,0,"rt <allcaps> <user>: ""light beer? you mean fag...",1,1,-1
3,3,2,rt <allcaps> <user>: my idea of a diet is eati...,1,1,1
4,3,2,rt <allcaps> <user>: <user> you're secretly a ...,1,1,1
...,...,...,...,...,...,...
681,3,2,rt <allcaps> <user>: this jimmy fallon prank o...,1,1,1
682,3,2,rt <allcaps> <user>: please win it all this ye...,1,1,1
683,3,0,fuck her pussy fuck her mouth fuck her ass cau...,1,1,-1
684,3,2,rt <allcaps> <user>: teixeira on point tonight...,1,1,1


In [23]:
hate_twits_evaluated['evaluation_soc'].describe()

count    686.0
mean       1.0
std        0.0
min        1.0
25%        1.0
50%        1.0
75%        1.0
max        1.0
Name: evaluation_soc, dtype: float64

In [28]:
hate_dict_socialsent.values()

dict_values([0.6833168472714036, 0.466564923431605, 0.7237276519549992, 0.7298797063358013, 0.7115301483236561, 0.7287661439644219, 0.7235634735799794, 0.5552279060650454, 0.2822738340682987, 0.7259811792911813, 0.6170247815476719, 0.639008031203559, 0.5881952013285665, 0.23318931616204558, 0.615339529186282, 0.6997557298120012, 0.7699835758865097, 0.7262080031969591, 0.2572710053942658, 0.7877430053350344, 0.5029530539346037, 0.5272375149782841, 0.727423281696214, 0.6394097855241145, 0.5713150111619552, 0.8366119907895717, 0.146586426573377, 0.7069466506214575, 0.5022630130100537, 0.7203765495839461, 0.1667024847940609, 0.5216761398625618, 0.706927032016293, 0.5254911720549981, 0.6770705319549255, 0.1964474989466585, 0.7145183346491926, 0.7910814873828861, 0.7803942305893982, 0.7569312559729541, 0.7856484148795163, 0.6697595853927378, 0.6787361300481846, 0.7116062514536143, 0.214882054244952, 0.6979403810541838, 0.735863418759665, 0.7813304012744301, 0.7278433097461969, 0.705141814717